In [26]:
import TSeriesPreproccesing as TSPP
import SequencePreprocessing as SP
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
import utils as utils
from tslearn.clustering import TimeSeriesKMeans
import os 
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [40]:
reload(TSPP)
reload(SP)
pd.set_option('display.max_columns', 100)

In [54]:
stockDataSet = TSPP.StockDataSet(['aapl','nvda','amd','msft'],start = '2022-01-01',interval = '1h')
# stockDataSet = TSPP.StockDataSet(['amd'],start = '2015-01-01',interval = '1d')
stockDataSet.create_dataset()
stockDataSet.create_features() 
target_cols = [col for col in stockDataSet.df.columns if 'sumPctChgclose_' in col and '-' not in col]
stockDataSet.create_y_targets(target_cols)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [55]:
n_steps = 20
stockSequence = stockDataSet.create_sequence_set(n_steps)
# stockSequence.dfs[0]
stockSequence.dfs[0].columns[np.isinf(stockSequence.dfs[0]).any()].tolist()

[]

In [56]:

stockSequence.train_test_split()
# print(stockSequence.training_dfs[0][['pctChgclose','sumPctChgclose+1','sumPctChgclose+2','sumPctChgclose+3','sumPctChgclose+4','sumPctChgclose+5','sumPctChgclose+6']].head(20))
stockSequence.scale_seq()
print(stockSequence.X_feature_sets[0].scaling_method)

ScalingMethod.SBSG


In [57]:
# # test_features = x_sbsg_feature_sets[0].cols
# test_features = ['close','sma10','ema20','ema50']
# test_indices = [stockSequence.X_feature_dict[feature] for feature in test_features]

# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
# x_values = np.arange(len(stockSequence.X_train[50,:,20]))
# for i in test_indices:
#     axes[0].plot(x_values,stockSequence.X_train[75,:,i])
#     axes[1].plot(x_values,stockSequence.X_train_scaled[75,:,i])

In [58]:
X_train, y_train = SP.SequenceElement.create_array(stockSequence.train_seq_elements,scaled = True)
X_test, y_test = SP.SequenceElement.create_array(stockSequence.test_seq_elements,scaled = True)


In [59]:
# cluster_features = ['sumPctChgclose_1','sumPctChgclose_3','sumPctChgclose_6']
cluster_features = ['sumPctChgclose_1','sumPctChgclose_3','sumPctChgclose_6']
cluster_indices = [stockSequence.X_feature_dict[feature] for feature in cluster_features]
X_cluster = utils.create_Cluster_Seq(X_train,cluster_indices)

In [60]:
import math
n_clusters = math.ceil(math.sqrt(len(X_cluster))) // 3
# A good rule of thumb is choosing k as the square root of the number of points in the training data set in kNN

km = TimeSeriesKMeans(n_clusters=n_clusters)

labels = km.fit_predict(X_cluster)

In [61]:
reload(utils)
threshold_factor = 1 # Adjust as necessary

X_cluster_adj, X_train_adj, y_train_adj, labels_adj = utils.remove_outliers(X_cluster,X_train,y_train,labels,km,threshold_factor)
display(X_cluster.shape)
display(X_train.shape)
display(y_train.shape)
display(X_cluster_adj.shape)
display(X_train_adj.shape)
display(y_train_adj.shape)
display(labels_adj.shape)

(9892, 20, 3)

(9892, 20, 49)

(9892, 6)

(8329, 20, 3)

(8329, 20, 49)

(8329, 6)

(8329,)

In [62]:
reload(utils)
utils.visualizeData(labels_adj,X_cluster_adj)


In [63]:
from keras.layers import RepeatVector, TimeDistributed

def create_model():
    # Encoder
    model_lstm = Sequential()
    
    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True, input_shape=(None, len(X_train_adj[0][0]))))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=100, activation='tanh',return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=100, activation='tanh'))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))
    
    # Repeat the encoder output (which is the last hidden state) 
    # for 'n' times where 'n' is the number of prediction steps
    model_lstm.add(RepeatVector(6))  # Assuming you are predicting for 6 steps

    # Decoder
    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))
    
    model_lstm.add(TimeDistributed(Dense(1)))  # Predict one value for each time step

    optimizer = Adam(learning_rate=0.001)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=1e-5)

    model_lstm.compile(optimizer=optimizer, loss="mse")
    return model_lstm

In [51]:
# display(X_train[0:1,:,-1])
X_cluster_test = utils.create_Cluster_Seq(X_test,cluster_indices)
# display(X_cluster_test.shape)

cluster_assignments = km.predict(X_cluster_test)


for lab in set(labels_adj): 
    filtered_x_train, filtered_y_train = utils.filter_clusters(labels_adj, lab, X_train_adj, y_train_adj)


    # values, counts = np.unique(cluster_assignments, return_counts=True)

    # # Display the results
    # for value, count in zip(values, counts):
    #     print(f"Value: {value}, Count: {count}")

    X_cluster_test_adj,X_test_adj, y_test_adj, labels_test_adj = utils.remove_outliers(X_cluster_test,X_test,y_test,cluster_assignments,km,threshold_factor)


    filtered_x_test, filtered_y_test = utils.filter_clusters(labels_test_adj, lab, X_test_adj, y_test_adj)
    
    # display(X_cluster_test_adj.shape)
    # display(filtered_x_test.shape)
    # display(filtered_y_test.shape)

    if len(filtered_x_test) == 0:
        continue

    # display(filtered_x_train.shape)
    model_lstm = create_model()
    print(filtered_x_test.shape)
    print(filtered_y_test.shape)

    y_train_reshaped = np.expand_dims(y_train, axis=-1)
    y_test_reshaped = np.expand_dims(y_test, axis=-1)
    
    model_lstm.fit(filtered_x_train, filtered_y_train, batch_size=30, epochs=500, verbose=1)

    
    
    predicted_stock_price = model_lstm.predict(filtered_x_test)
    predicted_stock_price = np.squeeze(predicted_stock_price, axis=-1)
    print(predicted_stock_price.shape)



    # predicted_stock_price = predictiveScaler.inverse_transform(predicted_stock_price)
    # filtered_y_test = predictiveScaler.inverse_transform(filtered_y_test)

    # Assuming predicted_stock_price has two columns
    results = pd.DataFrame({
        'predicted_1': predicted_stock_price[:, 0],
        'predicted_2': predicted_stock_price[:, 1],
        'predicted_3': predicted_stock_price[:, 2],
        'predicted_4': predicted_stock_price[:, 3],
        'predicted_5': predicted_stock_price[:, 4],
        'predicted_6': predicted_stock_price[:, 5],
        'real_1': filtered_y_test[:, 0],
        'real_2': filtered_y_test[:, 1],
        'real_3': filtered_y_test[:, 2],
        'real_4': filtered_y_test[:, 3],
        'real_5': filtered_y_test[:, 4],
        'real_6': filtered_y_test[:, 5]
    })

    # Assuming a correct prediction means both predicted features have the same sign as their corresponding real values
    results['same_1'] = ((results['predicted_1'] > 0) & (results['real_1'] > 0)) | ((results['predicted_1'] < 0) & (results['real_1'] < 0))
    results['same_2'] = ((results['predicted_2'] > 0) & (results['real_2'] > 0)) | ((results['predicted_2'] < 0) & (results['real_2'] < 0))
    results['same_3'] = ((results['predicted_3'] > 0) & (results['real_3'] > 0)) | ((results['predicted_3'] < 0) & (results['real_3'] < 0))
    results['same_4'] = ((results['predicted_4'] > 0) & (results['real_4'] > 0)) | ((results['predicted_4'] < 0) & (results['real_4'] < 0))
    results['same_5'] = ((results['predicted_5'] > 0) & (results['real_5'] > 0)) | ((results['predicted_5'] < 0) & (results['real_5'] < 0))
    results['same_6'] = ((results['predicted_6'] > 0) & (results['real_6'] > 0)) | ((results['predicted_6'] < 0) & (results['real_6'] < 0))
    accuracy1 = results['same_1'].sum() / len(results) * 100
    accuracy2 = results['same_2'].sum() / len(results) * 100
    accuracy3 = results['same_3'].sum() / len(results) * 100
    accuracy4 = results['same_4'].sum() / len(results) * 100
    accuracy5 = results['same_5'].sum() / len(results) * 100
    accuracy6 = results['same_6'].sum() / len(results) * 100

    # Construct the string
    output_string = (
        "Cluster Number: " + str(lab) +
        " \nAccuracy1D " + str(accuracy1) + " PredictedRet: " + str(results['predicted_1'].mean()) + " ActRet " + str(results['real_1'].mean() ) +
        " \nAccuracy2D " + str(accuracy2) + " PredictedRet: " + str(results['predicted_2'].mean()) + " ActRet " + str(results['real_2'].mean() ) +
        " \nAccuracy3D " + str(accuracy3) + " PredictedRet: " + str(results['predicted_3'].mean()) + " ActRet " + str(results['real_3'].mean() ) +
        " \nAccuracy4D " + str(accuracy4) + " PredictedRet: " + str(results['predicted_4'].mean()) + " ActRet " + str(results['real_4'].mean() ) +
        " \nAccuracy5D " + str(accuracy5) + " PredictedRet: " + str(results['predicted_5'].mean()) + " ActRet " + str(results['real_5'].mean() ) +
        " \nAccuracy6D " + str(accuracy6) + " PredictedRet: " + str(results['predicted_6'].mean()) + " ActRet " + str(results['real_6'].mean() ) +
        " Test set length: " + str(len(filtered_y_test)) + "\n"
    )

    # Write the string to a file
    with open('fuck.txt', 'a') as f:
        f.write(output_string)






2023-10-13 19:08:51.476012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-13 19:08:51.476909: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-13 19:08:51.477548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(15, 20, 49)
(15, 6)
Epoch 1/250


2023-10-13 19:08:52.331041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-13 19:08:52.332081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-13 19:08:52.332985: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5/5 [==============================] - 6s 45ms/step - loss: 3.2991
Epoch 2/250
5/5 [==============================] - 0s 44ms/step - loss: 2.0574
Epoch 3/250
5/5 [==============================] - 0s 44ms/step - loss: 1.7052
Epoch 4/250
5/5 [==============================] - 0s 44ms/step - loss: 1.6259
Epoch 5/250
5/5 [==============================] - 0s 44ms/step - loss: 1.4546
Epoch 6/250
5/5 [==============================] - 0s 45ms/step - loss: 1.2099
Epoch 7/250
5/5 [==============================] - 0s 44ms/step - loss: 1.2075
Epoch 8/250
5/5 [==============================] - 0s 44ms/step - loss: 1.1556
Epoch 9/250
5/5 [==============================] - 0s 43ms/step - loss: 1.0114
Epoch 10/250
5/5 [==============================] - 0s 44ms/step - loss: 1.0008
Epoch 11/250
5/5 [==============================] - 0s 44ms/step - loss: 0.9125
Epoch 12/250
5/5 [==============================] - 0s 44ms/step - loss: 1.0313
Epoch 13/250
5/5 [==============================] - 0s 44ms/s

2023-10-13 19:09:52.575691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-13 19:09:52.576648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-13 19:09:52.577324: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

(14, 20, 49)
(14, 6)
Epoch 1/250


2023-10-13 20:20:41.582287: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-13 20:20:41.583323: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-13 20:20:41.584174: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5/5 [==============================] - 6s 48ms/step - loss: 3.1056
Epoch 2/250
5/5 [==============================] - 0s 47ms/step - loss: 2.2512
Epoch 3/250
5/5 [==============================] - 0s 47ms/step - loss: 1.6218
Epoch 4/250
5/5 [==============================] - 0s 47ms/step - loss: 1.8049
Epoch 5/250
5/5 [==============================] - 0s 47ms/step - loss: 1.3041
Epoch 6/250
5/5 [==============================] - 0s 46ms/step - loss: 1.2770
Epoch 7/250
5/5 [==============================] - 0s 47ms/step - loss: 1.0114
Epoch 8/250
5/5 [==============================] - 0s 47ms/step - loss: 1.0479
Epoch 9/250
5/5 [==============================] - 0s 47ms/step - loss: 0.9392
Epoch 10/250
5/5 [==============================] - 0s 47ms/step - loss: 0.9621
Epoch 11/250
5/5 [==============================] - 0s 47ms/step - loss: 0.9875
Epoch 12/250
5/5 [==============================] - 0s 47ms/step - loss: 0.8563
Epoch 13/250
5/5 [==============================] - 0s 46ms/s

2023-10-13 20:21:43.357295: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-13 20:21:43.358502: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-13 20:21:43.359328: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 1s/step
(14, 6)
